In [ ]:
from geoband.API import *
# 버스정류장
GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')
GetCompasData('SBJ_2102_002', '2', '2.수원시_버스정류장별_승하차이력(1).csv')
GetCompasData('SBJ_2102_002', '3', '3.수원시_버스정류장별_승하차이력(2).csv')
GetCompasData('SBJ_2102_002', '4', '4.수원시_버스정류장별_승하차이력(3).csv')
GetCompasData('SBJ_2102_002', '5', '5.수원시_버스정류장별_승하차이력(4).csv')
GetCompasData('SBJ_2102_002', '6', '6.수원시_버스정류장별_승하차이력(5).csv')
GetCompasData('SBJ_2102_002', '7', '7.수원시_버스정류장별_노선현황.csv')

# 지하철역
GetCompasData('SBJ_2102_002', '8', '8.수원시_지하철역_위치정보.csv')
GetCompasData('SBJ_2102_002', '9', '9.수원시_지하철역별_이용현황(2017~2019).csv')

# 옥외광고물, 대기오염도, 주차장, 기상
GetCompasData('SBJ_2102_002', '10', '10.수원시_옥외광고물현황.csv')
GetCompasData('SBJ_2102_002', '11', '11.수원시_대기오염도_측정현황.csv')
GetCompasData('SBJ_2102_002', '12', '12.수원시_주차장현황.csv')
GetCompasData('SBJ_2102_002', '13', '13.수원시_기상데이터(2020).csv')

# 유동인구
GetCompasData('SBJ_2102_002', '14', '14.수원시_시간대별_유동인구(2020).csv')
GetCompasData('SBJ_2102_002', '15', '15.수원시_성연령별_유동인구(2020).csv')
GetCompasData('SBJ_2102_002', '16', '16.수원시_요일별_유동인구(2020).csv')

# 인구정보
GetCompasData('SBJ_2102_002', '17', '17.수원시_인구정보(고령)_격자.geojson')
GetCompasData('SBJ_2102_002', '18', '18.수원시_인구정보(생산가능)_격자.geojson')
GetCompasData('SBJ_2102_002', '19', '19.수원시_인구정보(유소년)_격자.geojson')

# 교통량 & 혼잡강도
GetCompasData('SBJ_2102_002', '20', '20.수원시_교통노드.geojson')
GetCompasData('SBJ_2102_002', '21', '21.수원시_교통링크.geojson')
GetCompasData('SBJ_2102_002', '22', '22.수원시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2102_002', '23', '23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
GetCompasData('SBJ_2102_002', '24', '24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
GetCompasData('SBJ_2102_002', '25', '25.수원시_평일_일별_혼잡시간강도_LV6.csv')

# 인도
GetCompasData('SBJ_2102_002', '26', '26.수원시_인도(2017).geojson')

# 건물
GetCompasData('SBJ_2102_002', '27', '27.수원시_도로명주소(건물).geojson')
GetCompasData('SBJ_2102_002', '28', '28.수`원시_건물연면적_격자.geojson')

# 경계
GetCompasData('SBJ_2102_002', '29', '29.수원시_법정경계(시군구).geojson')
GetCompasData('SBJ_2102_002', '30', '30.수원시_법정경계(읍면동).geojson')
GetCompasData('SBJ_2102_002', '31', '31.수원시_행정경계(읍면동).geojson')

# 지적도
GetCompasData('SBJ_2102_002', '32', '32.수원시_지적도.geojson')

# 라이브러리 불러오기

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import json
import random
import matplotlib.pylab as plt
%matplotlib inline

# MCLP를 위한 선형계획법 툴
from pulp import *

# 시각화 툴 : Pydeck
import pydeck as pdk

# 지리 데이터 전처리 툴 : Shapely
import shapely.speedups
shapely.speedups.enable()
from shapely.ops import unary_union
from shapely.geometry import Point, MultiLineString, mapping, shape

mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA'

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

## 수원시 법정경계
`base`는 모든 시각화에서 수원시 경계를 나타내는 데 사용될 예정

In [ ]:
suwon = gpd.read_file('30.수원시_법정경계(읍면동).geojson')
suwon['coordinates'] = suwon['geometry'].apply(lambda x : mapping(x)['coordinates'][0])
suwon.head()

# 수원 중심, 시각화에 사용될 예정
center = [37.2737612, 126.9955691] 

In [ ]:
base = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    suwon, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.005
)

# EDA
`pydeck`은 geometry 타입을 받지 못하므로 인식가능한 형태로 변환이 필요하다.(`coordinates`)

## 1. 건물 시각화
### 정류장 주변의 취약계층 주요 이용시설(ex. 병원, 유치원 등)
---

- 전처리

In [ ]:
# 같은 건물임에도 서로 다른 행에 존재해서 count할 때 중복되어 집계되므로
# QGIS를 사용해 같은 지번, 건물타입을 갖는 건물을 하나의 건물로 묶음
building_union = gpd.read_file('building_union.geojson')
building_union['coordinates'] = building_union['geometry'].apply(lambda x : mapping(x)['coordinates'][0])
building_union.head()

In [ ]:
# 코드 정의서를 참고하여 정의한 
# 취약계층 주요 이용시설건물코드, 미세먼지 유발시설 건물코드
vulnerable_codes = ['03018', '03108', '07000', '07101', '07102', '07104', '07107', '07999', '08101', 
         '08102', '08103', '08201', '08202', '08203', '08204', '08299', '08300', '21003']
dust_codes = ['06304', '06305', '13000', '13100', '13200', '16006', '19004', '18002', '18003']

def func(x):
    if x not in vulnerable_codes+dust_codes:
        return 0
    else:
        return 200
building_union['value'] = building_union['BDTYP_CD'].apply(lambda x: func(x))

- 시각화
- 취약계층 주요 이용시설(Vulnerable_code)

In [ ]:
layer1 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    building_union[building_union['BDTYP_CD'].apply(lambda x: x in vulnerable_codes)] , # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    filled=True,
    extruded=True,
    wireframe=True,
    get_fill_color='[0,255,0]',
    get_elevation="value",
    pickable=True, auto_highlight=True
)

center = center
view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1], 
    zoom=12,
    pitch = 50
) 

r = pdk.Deck(layers=[base, layer1], initial_view_state=view_state,
             mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA') 
r.to_html()

- 미세먼지 유발시설

In [ ]:
layer2 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    building_union[building_union['BDTYP_CD'].apply(lambda x: x in dust_codes)] , # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0,255,0]',
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="value",
    pickable=True, auto_highlight=True
)

center = center
view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1], 
    zoom=12,
    pitch = 50
) 

r = pdk.Deck(layers=[base, layer2], initial_view_state=view_state,
             mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA') 
r.to_html()

## 2. 성연령별 유동인구 시각화( 취약계층과 총 유동인구의 분포 비교)
---

In [ ]:
pop_gender = pd.read_csv('15.수원시_성연령별_유동인구(2020).csv').groupby(['lon', 'lat']).mean()
pop_gender['TOTAL'] = pop_gender.sum(axis = 1)
sns.heatmap(pop_gender.drop('STD_YM', axis=1).corr(),cmap='YlGnBu')

- 전 연령층 전처리

In [ ]:
# 좌표 별 유동인구 합
pop_gender = pd.read_csv('15.수원시_성연령별_유동인구(2020).csv')
# 좌표별로 그룹화 진행 시간별로 더해줌
pop_gender = pop_gender.groupby(['lon', 'lat']).sum() 
pop_gender = pop_gender.drop(['STD_YM'], axis = 1)
# 모든 성연령별 유동인구 합계
pop_gender['TOTAL'] = pop_gender.sum(axis = 1)
pop_gender = pop_gender.reset_index()
# df -> gdf
pop_gender = gpd.GeoDataFrame(pop_gender, 
                              geometry=gpd.points_from_xy(pop_gender.lon, pop_gender.lat))
pop_gender.head(3)

- 취약계층(10대 미만, 60대 이상) 전처리

In [ ]:
# 각 날짜별 10대인구, 60대 이상인구 각 좌표에서 시간별로 sum
pop_gender_vul = pd.read_csv('15.수원시_성연령별_유동인구(2020).csv')[['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU',
                                                            'MAN_FLOW_POP_CNT_10G','WMAN_FLOW_POP_CNT_10G','lon','lat','STD_YM']]
pop_gender_vul = pop_gender_vul.groupby(['lon', 'lat']).sum()
# 불필요한 column 제거
pop_gender_vul = pop_gender_vul.drop(['STD_YM'], axis = 1)
# 성별 관계없이 연령별로 합계
pop_gender_vul['TOTAL_10'] = pop_gender_vul[['WMAN_FLOW_POP_CNT_10G','MAN_FLOW_POP_CNT_10G']].sum(axis = 1)
pop_gender_vul['TOTAL_60'] = pop_gender_vul[['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU']].sum(axis = 1)

# df -> gdf
pop_gender_vul = pop_gender_vul.reset_index()
pop_gender_vul = gpd.GeoDataFrame(pop_gender_vul, geometry=gpd.points_from_xy(pop_gender_vul.lon, pop_gender_vul.lat))
pop_gender_vul = pop_gender_vul.drop(['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU',
                                      'WMAN_FLOW_POP_CNT_10G','MAN_FLOW_POP_CNT_10G','lon', 'lat'], axis=1)
pop_gender_vul['lon'] = pop_gender_vul['geometry'].apply(lambda x : mapping(x)['coordinates'][0])
pop_gender_vul['lat'] = pop_gender_vul['geometry'].apply(lambda x : mapping(x)['coordinates'][1])
pop_gender_vul['TOTAL_vul'] = pop_gender_vul[['TOTAL_10','TOTAL_60']].sum(axis = 1)
pop_gender_vul['TOTAL'] = pop_gender['TOTAL']
pop_gender_vul.head(3)

- 시각화

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  #pop_gender_vul[pop_gender_vul['TOTAL_vul']> np.quantile(pop_gender_vul['TOTAL_vul'],0.8)][pop_gender_vul['TOTAL_vul']/pop_gender_vul['TOTAL'] > np.quantile(pop_gender_vul['TOTAL_vul']/pop_gender_vul['TOTAL'], 0.95)], 
                  pop_gender_vul[pop_gender_vul['TOTAL_vul'] > np.quantile(pop_gender_vul['TOTAL_vul'], 0.9)],
                  get_position = ['lon','lat'], 
                  get_fill_color='[255,0,0]',
                  opacity = 0.05,
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
scat2 = pdk.Layer('ScatterplotLayer',
                  pop_gender_vul[pop_gender['TOTAL'] > np.quantile(pop_gender['TOTAL'], 0.9)], 
                  get_position = ['lon','lat'],
                  get_fill_color='[0,0,255]',
                  get_radius=40,
                  opacity = 0.1,
                  pickable=True, auto_highlight=True 
                 ) 

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
r = pdk.Deck(layers=[base, scat1, scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 3. 시간대별 유동인구 시각화

In [ ]:
# 유동인구 월별, 시간별 linePlot
# 유동인구는 날짜에 큰 관계없이 시간별로 일정한 특징이 존재함
pop_time = pd.read_csv('14.수원시_시간대별_유동인구(2020).csv')
pop = pop_time.groupby(['STD_YM']).mean().reset_index()
pop = pop.drop(['lon', 'lat'], axis=1)

x = list(range(0,24))
for i in range(len(pop)):
    plt.plot(x, pop.iloc[i, 1:25].values)
    
plt.title('Population by time zone', fontsize=18) 
plt.ylabel('Population', fontsize=14)
plt.xlabel('Time(Hour)', fontsize=14)
plt.legend(list(map(lambda x: "Month: "+str(x), range(1,13))), loc='best')

plt.show()

In [ ]:
# 시간대별 분포를 확인하기 위해 좌표별로 평균을 구함
test = pop_time.groupby(['lon','lat']).mean()
test = test.drop(['STD_YM'], axis = 1).reset_index()
test_lonlat = test[['lon', 'lat']]
test = test.drop(['lon', 'lat'], axis=1)
test.head(3)

In [ ]:
# 유동인구가 많은 지역을 파악하기 위해 70퍼센트 quantile 값으로 필터링
# 시각화를 용이하게 하기위해 0부터 1까지의 값을 가지도록 스케일링
# 이후에 지리적 차이를 확인하는 시각화에서 대부분 이와 같은 스케일링 과정을 거침 
maximum = test.max().max()
q70 = np.quantile(test,0.7)

# 6시간 간격으로 유동인구의 변화를 확인
test['TMST_06_reg'] = test['TMST_06']/maximum
test['TMST_12_reg'] = test['TMST_12']/maximum
test['TMST_18_reg'] = test['TMST_18']/maximum
test['TMST_00_reg'] = test['TMST_00']/maximum

test = pd.concat([test, test_lonlat], axis=1)
test.head(3)

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  test[test['TMST_06']>q70], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,5000*TMST_06_reg,200]',
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 
scat2 = pdk.Layer('ScatterplotLayer',
                  test[test['TMST_12']>q70], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,5000*TMST_12_reg,200]',
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 
scat3 = pdk.Layer('ScatterplotLayer',
                  test[test['TMST_18']>q70], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,5000*TMST_18_reg,200]',
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 
scat4 = pdk.Layer('ScatterplotLayer',
                  test[test['TMST_00']>q70], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,5000*TMST_00_reg,200]',
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 
center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=10
) 

### 06시 유동인구 시각화
r = pdk.Deck(layers=[base,scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
### 12시 유동인구 시각화
r = pdk.Deck(layers=[base,scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
### 18시 유동인구 시각화
r = pdk.Deck(layers=[base,scat3], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
### 00시 유동인구 시각화
r = pdk.Deck(layers=[base,scat4], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 4. 시간대별 상세 도로의 추정 교통량 시각화

- 전처리

In [ ]:
# 시간대별 교통량
traffic = pd.read_csv('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
traffic = traffic.rename(columns = {'상세도로망_LinkID' : 'link_id'})

# 추정교통량의 link_id는 상행/하행정보를 포함하고 있어 매핑이 되지 않으므로 분리해줌
traffic['link_id'] = traffic['link_id'].apply(lambda x : str(x))
traffic['상행하행'] = traffic['link_id'].apply(lambda x: x[-2:])
traffic['link_id'] = traffic['link_id'].apply(lambda x: x[:-2])

# 하루동안의 교통량의 분포를 파악하기 위해 fulltime 분리
traffic_fulltime = traffic[traffic['시간적범위'] == 'fulltime']
network = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
traffic_fulltime['link_id'] = traffic_fulltime['link_id'].apply(lambda x : str(x))
traffic_fulltime = traffic_fulltime.merge(network, on =['link_id'], how='left')
traffic_fulltime['전체_추정교통량_reg'] = traffic_fulltime['전체_추정교통량']/traffic_fulltime['전체_추정교통량'].max()
traffic_fulltime['coordinates'] = traffic_fulltime['geometry'].apply(lambda x : mapping(x)['coordinates'][0])

traffic_ver1 = traffic[traffic['시간적범위'] != 'fulltime']

traffic_ver1['시간적범위'] = traffic_ver1['시간적범위'].apply(lambda x : int(x))
traffic_mean = traffic_ver1.groupby(['시간적범위']).mean()
traffic_mean.head()

In [ ]:
plt.plot(traffic_mean.index.values, traffic_mean.전체_추정교통량.values, color='green')
    
plt.title('Traffic by time zone', fontsize=18) 
plt.ylabel('Traffic', fontsize=14)
plt.xlabel('Time(Hour)', fontsize=14)

plt.show()

In [ ]:
# 교통량과 상세도로망, link_id 기준으로 병합
network = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
traffic_ver2 = traffic_ver1.merge(network, on =['link_id'], how='left')
traffic_ver2.head(3)

In [ ]:
p70 = np.percentile(traffic_ver2['전체_추정교통량'], 70)

traffic_ver2['전체_추정교통량_reg'] = traffic_ver2['전체_추정교통량'].apply(lambda x: x/p70 if x<=p70 else 1)
traffic_ver2['coordinates'] = traffic_ver2['geometry'].apply(lambda x : mapping(x)['coordinates'][0])

In [ ]:
traffic_00 = traffic_ver2[traffic_ver2['시간적범위']==0]
traffic_06 = traffic_ver2[traffic_ver2['시간적범위']==6]
traffic_12 = traffic_ver2[traffic_ver2['시간적범위']==12]
traffic_18 = traffic_ver2[traffic_ver2['시간적범위']==18]

- 시각화

- 00시 시각화

In [ ]:
layer1 = pdk.Layer( 'PathLayer', 
                  traffic_00, 
                  get_path='coordinates', 
                  get_width='전체_추정교통량/50', 
                  get_color='[255,255*전체_추정교통량_reg,120]', 
                  pickable=True, auto_highlight=True 
                 )
layer2 = pdk.Layer( 'PathLayer', 
                  traffic_06, 
                  get_path='coordinates', 
                  get_width='전체_추정교통량/50', 
                  get_color='[255,255*전체_추정교통량_reg,120]', 
                  pickable=True, auto_highlight=True 
                 ) 
layer3 = pdk.Layer( 'PathLayer', 
                  traffic_12, 
                  get_path='coordinates', 
                  get_width='전체_추정교통량/50', 
                  get_color='[255,255*전체_추정교통량_reg,120]', 
                  pickable=True, auto_highlight=True 
                 ) 
layer4 = pdk.Layer( 'PathLayer', 
                  traffic_18, 
                  get_path='coordinates', 
                  get_width='전체_추정교통량/50', 
                  get_color='[255,255*전체_추정교통량_reg,120]', 
                  pickable=True, auto_highlight=True 
                 ) 

center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=10
) 
r = pdk.Deck(layers=[base, layer1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

- 06시 교통량 시각화

In [ ]:
r = pdk.Deck(layers=[base, layer2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

- 12시 교통량 시각화

In [ ]:
r = pdk.Deck(layers=[base, layer3], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

- 18시 교통량 시각화

In [ ]:
r = pdk.Deck(layers=[base, layer4], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 5. 대기오염도(미세먼지) - 미세먼지 대체 변수로 활용하기 위한 시각화

In [ ]:
from sklearn.preprocessing import StandardScaler

air = pd.read_csv('11.수원시_대기오염도_측정현황.csv')
air = air.rename(columns = {'미세먼지(㎍/㎥)' : '미세먼지', '초미세먼지(㎍/㎥)' : '초미세먼지','오존(ppm)':'오존',
                            '이산화질소(ppm)': '이산화질소', '아황산가스(ppm)' : '아황산가스', '일산화탄소(ppm)' : '일산화탄소'})
air = air[air['미세먼지'] != '-'] #결측값 제거
air['미세먼지'] = air['미세먼지'].apply(pd.to_numeric)
air['초미세먼지'] = air['초미세먼지'].apply(pd.to_numeric)

# 오염물질 측정단위 통일
air['오존'] = air['오존'].apply(pd.to_numeric)*(48/22.4)*1000
air['이산화질소'] = air['이산화질소'].apply(pd.to_numeric)*(46.0055/22.4)*1000
air['아황산가스'] = air['아황산가스'].apply(pd.to_numeric)*(64.006/22.4)*1000
air['일산화탄소'] = air['일산화탄소'].apply(pd.to_numeric)*(28.01/22.4)*1000

# 스케일링 진행, anova 테스트에는 영향을 주지 않음
dust = ['미세먼지', '초미세먼지', '오존', '이산화질소', '아황산가스', '일산화탄소']
scaler = StandardScaler()
air[dust] = scaler.fit_transform(air[dust])

air.head(3)

In [ ]:
import scipy.stats as stats

# anova
for d in dust:
    g1, g2, g3, g4, g5, g6, g7, g8 = [air[air['측정소']==a][d].values for a in air.측정소.unique()]
    F_statistic, pVal = stats.f_oneway(g1, g2, g3, g4, g5, g6, g7, g8)
    if pVal < 0.05:
        print(f"<{d}> F-value : {round(F_statistic, 3)}, p-value : {round(pVal,3)} ***")
    else:
        print(f"<{d}> F-value : {round(F_statistic, 3)}, p-value : {round(pVal,3)}")

# 일산화탄소와 이산화질소 : 귀무가설 기각    
# 일산화탄소와 이산화질소의 주 배출원은 자동차(도로이동오염원)!

In [ ]:
air = air.groupby(['측정소']).mean()
air['coordinates'] = list(zip(air.lon, air.lat))
air.head()

In [ ]:
air['dust_이산화질소'] = air['이산화질소'] - air['이산화질소'].min()/(air['이산화질소'].max() - air['이산화질소'].min())
air['dust_일산화탄소'] = air['이산화질소'] - air['이산화질소'].min()/(air['이산화질소'].max() - air['이산화질소'].min())
air.head()

In [ ]:
layer = pdk.Layer( 'PathLayer', 
                  traffic_fulltime, 
                  get_path='coordinates', 
                  get_width='전체_추정교통량/150', 
                  get_color='[255,255*전체_추정교통량_reg,120]', 
                  pickable=True, auto_highlight=True 
                 ) 

scat1 = pdk.Layer(
    "ScatterplotLayer",
    air,
    get_position = ['lon','lat'],
    get_color = '[255,255,255]',
    get_fill_color = '[0,500*dust_이산화질소,255]',
    get_radius= 200,
    pickable=True, auto_highlight=True
)

scat2 = pdk.Layer(
    "ScatterplotLayer",
    air,
    get_position = ['lon','lat'],
    get_color = '[255,255,255]',
    get_fill_color = '[0,500*dust_일산화탄소,255]',
    get_radius= 200,
    pickable=True, auto_highlight=True
)

center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=10
) 

#view_state.bearing = -15
#view_state.pitch = 45

r = pdk.Deck(layers=[base,layer, scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
r = pdk.Deck(layers=[base,layer, scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

# 모델링을 위한 전처리 과정

## 인도 폭 & 버스 정류장 시각화

- 전처리

In [ ]:
from shapely.geometry import MultiLineString, mapping, shape

india = gpd.read_file('26.수원시_인도(2017).geojson')
india['coordinates'] = india['geometry'].apply(lambda x : mapping(x)['coordinates'][0])

india['WIDT_reg'] = india['WIDT'] / india['WIDT'].max() # 시각화를 위한 정규화
india.head(3)

- 시각화
- 버스정류장과 인도

In [ ]:
station_raw = pd.read_csv('1.수원시_버스정류장.csv')

layer1 = pdk.Layer( 'PathLayer', 
                  india, 
                  get_path='coordinates', 
                  get_color='[255,165,0]', 
                  width_min_pixels=2,
                  pickable=True, auto_highlight=True 
                 ) 

scat1 = pdk.Layer('ScatterplotLayer',
                  station_raw, 
                  get_position = ['lon','lat'], 
                  get_color = '[255,255,255]',
                  get_fill_color='[102,160,145]',
                  get_radius=40,
                  pickable=True, auto_highlight=True 
                 ) 

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, layer1, scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 버스정류장 전처리
- QGIS를 이용해 정류장(`1.수원시_버스정류장.csv`)과 그것에 최근접한 인도(`26.수원시_인도(2017).geojson`)를 매핑 후 `read_csv`

In [ ]:
# 총 1179개의 버스정류장
station = pd.read_csv('station_sidewalk.csv').iloc[:, :-6]
station = station.drop(['UFID', 'QUAL', "BYYN", 'KIND','쉘터','LED',
                        'LCD','LED+LCD복합형', '알뜰형','임대형(음성)'], axis=1)

# 인도폭이 4m 이상(최소 보도폭 : 1.5m + 스마트 버스정류장의 세로길이 : 2.5m)
station = station[station['WIDT'] >= 4]

# BIS가 설치된 곳은 전기/통신 설비를 갖춤
station= station.dropna(subset = ['BIS설치여부'])
print(station.shape)
station.rename(columns = {'정류장ID':'정류소ID'}, inplace=True)
station.head(3)

#### 수원시의 버스정류장, 인도폭과 BIT설치여부로 필터링된 정류장(분석대상) 시각화

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  station_raw, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[102,160,145]',
                  get_radius=50,
                  pickable=True, auto_highlight=True 
                 ) 

scat2 = pdk.Layer('ScatterplotLayer',
                  station, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[152,255,152]',
                  get_radius=120,
                  pickable=True, auto_highlight=True 
                 ) 

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat1, scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 승하차 이력 합치기
- 정류소별 전체 승차 건수 & 환승 건수 & 전체 하차 건수

In [ ]:
data = ['2.수원시_버스정류장별_승하차이력(1).csv', '3.수원시_버스정류장별_승하차이력(2).csv', '4.수원시_버스정류장별_승하차이력(3).csv', 
          '5.수원시_버스정류장별_승하차이력(4).csv', '6.수원시_버스정류장별_승하차이력(5).csv']

df = pd.concat([pd.read_csv(d) for d in data])

# 환승 건수는 전체 승차 건수에 포함되어 있음
# 각 정류소의 일별 승, 하차 평균
on_off = df.groupby(['정류소ID'])[['전체 승차 건수','환승 건수', '전체 하차 건수']].mean()
on_off.reset_index(inplace = True)
# bus_station과 인덱스 명을 맞춰줌
#on_off = on_off.rename(columns = {'정류소ID' : '정류장ID'})
on_off.head()

In [ ]:
# 정류소별 운행하는 버스 종류에 따라 평균 승차인원
bus_route = df[['정류소ID','노선번호', '노선유형','전체 승차 건수']]
bus_route['노선번호'] = bus_route['노선번호'].apply(lambda x: str(x).replace('_','-'))
bus_route['노선번호'] = bus_route['노선번호'].apply(lambda x: str(x).replace('예약',''))

bus_route = bus_route.groupby(['정류소ID', '노선유형'])['전체 승차 건수'].mean()
bus_route = bus_route.unstack(fill_value = 0)

# 맞춤형시내버스 == 따복형시내버스
bus_route_list = ['경기외곽순환버스', '광역급행형시내버스', '따복형시내버스',
                  '일반형시내버스','좌석형시내버스', '직행좌석형시내버스']
bus_route.columns = bus_route_list
bus_route = bus_route.reset_index()
bus_route.head()

In [ ]:
# 버스정류장에 승하차, 노선 정보 병합
station_1 = station.merge(on_off, on='정류소ID', how='left')
station_ver1 = station_1.merge(bus_route, on = '정류소ID', how='left').fillna(0)
station_ver1 = gpd.GeoDataFrame(station_ver1, geometry=gpd.points_from_xy(station_ver1.lon, station_ver1.lat))
station_ver1.head(1)

## 버스 노선별 배차간격 (공공데이터 포털 API 이용)
---

In [ ]:
import xmltodict
import json
from urllib import parse, request

station_raw = pd.read_csv('1.수원시_버스정류장.csv')
station_raw.head(1)

- 수원시의 정류장을 지나다니는 모든 노선의 ID를 추출
- 버스 도착 시간 정보 API : 입력 값 : 정류장 ID

In [ ]:
url = 'http://openapi.gbis.go.kr/ws/rest/busarrivalservice/station'
API_KEY = 'xDCJxBLtrjyz7m6c+/BAENwUEKfqE5/nlSnSeyutrm/P8jezX9lpPg2Ej9iemgcMcy9sNtA91wxnSsqmc1Mx6A=='

routeid_list = set()

for station_id in tqdm(list(map(str, station_raw['정류장ID'].values))):
    queryParams = '?' + parse.urlencode({ parse.quote_plus('ServiceKey') : API_KEY, parse.quote_plus('serviceKey') : '1234567890', parse.quote_plus('stationId') : station_id })

    requests = request.Request(url + queryParams)
    requests.get_method = lambda: 'GET'
    response_body = request.urlopen(requests).read()

    result = xmltodict.parse(response_body)
    stationid_info = json.loads(json.dumps(result))
    
    try:
        routeid = pd.DataFrame(stationid_info['response']['msgBody']['busArrivalList'])['routeId']
        routeid_list.update(routeid.values)
    except:
        continue
    
# 5개만 랜덤하게 추출해서 확인
set(random.sample(routeid_list, 5))

- 추출한 노선의 ID를 추출하여 그 값을 입력값으로 하여 노선유형별 배차간격 추출
- 버스 노선 정보 API : 입력값 : 노선ID 

In [ ]:
from collections import defaultdict

url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
API_KEY = 'xDCJxBLtrjyz7m6c+/BAENwUEKfqE5/nlSnSeyutrm/P8jezX9lpPg2Ej9iemgcMcy9sNtA91wxnSsqmc1Mx6A=='
route_dict = dict()

for rooteid in tqdm(routeid_list):
    temp = url + '?ServiceKey=' + API_KEY + '&serviceKey=1234567890&routeId=' + rooteid

    requests = request.Request(temp)
    requests.get_method = lambda: 'GET'
    response_body = request.urlopen(requests).read()
    result_1 = xmltodict.parse(response_body)
    routeId_info = json.loads(json.dumps(result_1))
    try:
        route_type = routeId_info['response']['msgBody']['busRouteInfoItem']['routeTypeName']
        min_interval = routeId_info['response']['msgBody']['busRouteInfoItem']['peekAlloc']
        max_interval = routeId_info['response']['msgBody']['busRouteInfoItem']['nPeekAlloc']
        route_dict[rooteid] = [route_type, np.mean([int(min_interval), int(max_interval)])]
    except:
        continue

In [ ]:
route_interval = pd.DataFrame(route_dict).transpose()
route_interval.columns = ['노선유형', '배차시간']
route_interval = route_interval[route_interval['노선유형'] != '일반형 공항버스']
route_interval = route_interval[route_interval['노선유형'] != '리무진형 공항버스']
route_interval = route_interval[route_interval['노선유형'] != '일반형시외버스']

route_interval['배차시간'] = route_interval['배차시간'].apply(pd.to_numeric)
route_interval  = route_interval[route_interval['배차시간'] < 60] # 이상치 제거

wait_time = route_interval.groupby(['노선유형']).median()/2
wait_time = wait_time.reset_index()

wait_time.rename(columns = {'배차시간': '대기시간'}, inplace = True)
wait_time

In [ ]:
# API에 따라 결과가 계속 바뀌어서 2021.03.28 시점 기준으로 데이터 저장
#wait_time.to_csv('wait_time.csv', index=False)
wait_time = pd.DataFrame({'노선유형':['경기외곽순환버스', '광역급행형시내버스','따복형시내버스',
                                 '일반형시내버스','좌석형시내버스','직행좌석형시내버스'],
                        '대기시간':[27.5 ,9.375 ,27.5 ,8 , 5.5, 10.625]})
wait_time

| | 노선유형 | 대기시간|
|---|---|---|
| 0 | 경기외곽순환버스 | 27.500 |
| 1 | 광역급행형시내버스 | 9.375 |
| 2 | 따복형시내버스 | 27.500 |
| 3 | 일반형시내버스 | 8.000 |
| 4 | 좌석형시내버스 | 5.500 |
| 5 | 직행좌석형시내버스 | 10.625 |


In [ ]:
# 버스유형마다 각 유형에 해당하는 대기시간을 곱해줌
for route_type, val in wait_time.values:
    if route_type not in bus_route_list:
        val = np.mean(wait_time.대기시간)
    station_ver1[route_type] = station_ver1[route_type] * val

# type_list
type_list = wait_time.노선유형.values

station_ver1['총대기시간'] = station_ver1[type_list].sum(axis=1)
station_ver1.drop(bus_route_list, axis=1, inplace=True)
station_ver1.head(1)

## 정류장 버퍼 폴리곤 생성
정류장을 중심으로 radius meter만큼 반지름을 갖는 원 폴리곤
<br>`정류소별 영향권 및 접근거리를 반영한 버스 통행배정 신뢰성 향상 방안 연구`에 따라 버스 정류장의 영향권은 400m로 설정합니다.

In [ ]:
# radius : 미터
def make_circle_polygon(gdf, radius = 400):
    
    gdf.crs = 'epsg:4326'
    gdf = gdf.to_crs(epsg=5179) 

    buffer = gdf.buffer(radius)
    gdf['BUFFER'] = buffer.to_crs(epsg=4326)
    gdf = gdf.to_crs(epsg=4326)
    return gdf
    
station_ver1 = make_circle_polygon(station_ver1)
station_ver1.head(1)

In [ ]:
station_ver1['coordinates'] = station_ver1['BUFFER'].apply(lambda x : mapping(x)['coordinates'][0])

In [ ]:
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    station_ver1, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 0, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.1
)

scat = pdk.Layer('ScatterplotLayer',
                  station_ver1, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,255,0]',
                  get_radius ='70',
                  pickable=True, auto_highlight=True 
                 ) 

center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=10
) 
r = pdk.Deck(layers=[base, layer, scat], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 건물 전처리
### 정류장 주변의 취약계층 주요 이용시설(ex. 병원) & 정류장 주변의 미세먼지 발생 시설
---

In [ ]:
# 같은 건물임에도 서로 다른 행에 존재해서 count할 때 중복되어 집계되므로
# QGIS를 사용해 같은 지번, 건물타입을 갖는 건물을 하나의 건물로 묶음
from shapely.ops import unary_union
building_union = gpd.read_file('building_union.geojson')
building_union['coordinates'] = building_union['geometry'].apply(lambda x : mapping(x)['coordinates'][0])
building_union.head()

In [ ]:
# 정류장 버퍼 내에 취약계층이 주로 이용하는 시설과 미세먼지를 유발하는 시설의 수를 얻는다.
vulnerable_codes = ['03018', '03108', '07000', '07101', '07102', '07104', '07107', '07999', '08101', 
         '08102', '08103', '08201', '08202', '08203', '08204', '08299', '08300', '21003']

# {col}값이 {code}와 같은 {df}의 행들 중에서, {poly}와 겹치는 행(시설)의 개수 반환
def intersect_buffer(df, col ,code, poly):
    df = df[df[col] == code]
    in_out = df.intersects(poly)
    within_buffer = df[in_out]
    return len(within_buffer)

# 반복문을 돌며 {_codes}에 포함되는 시설 개수를 표시하는 칼럼 추가
print('-----취약계층-----')
for c in vulnerable_codes:
    print(f'{c}가 처리되는중..')
    station_ver1[c] = station_ver1['BUFFER'].apply(lambda x: intersect_buffer(building_union, 'BDTYP_CD', c, x))

vulnerable_dict = { '03018' : '조산원', '03108' : '보건소', '07000' : '의료시설', '07101' : '종합병원',
                   '07102' : '산부인과병원', '07104' : '한방병원', '07107' : '병원', '07999' : '기타의료시설',
                   '08101' : '초등학교', '08102' : '중학교', '08103' : '고등학교', '08201' : '유치원',
                   '08202' : '영유아보육시설' , '08203' : '어린이집', '08204' : '아동복지시설',
                   '08299' : '기타아동관련시설', '08300' : '노인복지시설', '21003' : '어린이회관'}    
station_ver1 = station_ver1.rename(columns = vulnerable_dict)


dust_codes = ['06304', '06305', '13000', '13100', '13200', '16006', '19004', '18002', '18003', '19004']

print('----미세먼지유발----')
for c in dust_codes:
    print(f'{c}가 처리되는중..')
    station_ver1[c] = station_ver1['BUFFER'].apply(lambda x: intersect_buffer(building_union, 'BDTYP_CD', c, x))

dust_dict = { '06304' : '화물터미널', '06305' : '철도역사', '13000' : '공장', '13100' : '일반공장',
            '13200' : '공해공장', '16006' : '정비공장', '19004' : '발전소', '18002' : '폐기물처리시설','18003' : '폐기물재활용시설'}
station_ver2 = station_ver1.rename(columns = dust_dict)
station_ver2.head(1)

In [ ]:
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    station_ver1, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 0, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.1
)

scat = pdk.Layer('ScatterplotLayer',
                  station_ver2, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,255,0]',
                  get_radius ='70',
                  pickable=True, auto_highlight=True 
                 ) 

layer1 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    building_union[building_union['BDTYP_CD'].apply(lambda x: x in vulnerable_codes)] , # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[200, 0, 0]',
    pickable=True, auto_highlight=True
)

layer2 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    building_union[building_union['BDTYP_CD'].apply(lambda x: x in dust_codes)] , # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 0, 200]',
    pickable=True, auto_highlight=True
)

center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=12
) 
r = pdk.Deck(layers=[base, layer, scat, layer1, layer2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
station_ver3 = station_ver2.drop(list(station_ver2.iloc[:,-27:].columns), axis=1)
station_ver3['취약계층건물'] = station_ver2[vulnerable_dict.values()].sum(axis=1)
station_ver3['먼지유발건물'] = station_ver2[dust_dict.values()].sum(axis=1)
station_ver3.head(2)

## 성연령별 유동인구 전처리
-  취약 계층

In [ ]:
# 동일한 위도 경도 위치 기준으로 있는 각 날짜별 인구수 sum
pop_gender_vul = pd.read_csv('15.수원시_성연령별_유동인구(2020).csv')[['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_10G','WMAN_FLOW_POP_CNT_10G','lon','lat','STD_YM']].groupby(['lon', 'lat']).sum()
# 불필요한 column 제거
pop_gender_vul = pop_gender_vul.drop(['STD_YM'], axis = 1)
# 동일한 위도 경도 위치에 있는 취약 계층 빈도를 sum
pop_gender_vul['TOTAL_10'] = pop_gender_vul[['WMAN_FLOW_POP_CNT_10G','MAN_FLOW_POP_CNT_10G']].sum(axis = 1)
pop_gender_vul['TOTAL_60'] = pop_gender_vul[['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU']].sum(axis = 1)
pop_gender_vul = pop_gender_vul.reset_index()

pop_gender_vul = gpd.GeoDataFrame(pop_gender_vul, geometry=gpd.points_from_xy(pop_gender_vul.lon, pop_gender_vul.lat))
pop_gender_vul = pop_gender_vul.drop(['WMAN_FLOW_POP_CNT_60GU','MAN_FLOW_POP_CNT_60GU',
                                      'WMAN_FLOW_POP_CNT_10G','MAN_FLOW_POP_CNT_10G','lon', 'lat'], axis=1)
pop_gender_vul.head()

In [ ]:
pop_gender_vul['lon'] = pop_gender_vul.geometry.x
pop_gender_vul['lat'] = pop_gender_vul.geometry.y

In [ ]:
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    station_ver1, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 0, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.1
)

scat1 = pdk.Layer('ScatterplotLayer',
                  station_ver1, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,255,0]',
                  get_radius ='70',
                  pickable=True, auto_highlight=True 
                 ) 

scat2 = pdk.Layer('ScatterplotLayer',
                  pop_gender_vul, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[170, 249, 209]',
                  get_radius ='10',
                  pickable=True, auto_highlight=True 
                 ) 

center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=12
) 
r = pdk.Deck(layers=[base, scat2, layer, scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

In [ ]:
# 반경 내에 취약계층(10대, 60대) 유동인구 평균 추가
from shapely.geometry import Point

def pop_within_buffer(df, column, poly):
    in_out = df.within(poly)
    within_buffer = df[in_out]
    return np.mean(within_buffer[column])

station_ver3['POP_10'] = station_ver3['BUFFER'].apply(lambda x: pop_within_buffer(pop_gender_vul, 'TOTAL_10', x))
station_ver3['POP_60'] = station_ver3['BUFFER'].apply(lambda x: pop_within_buffer(pop_gender_vul, 'TOTAL_60', x))
station_ver3['POP_vul'] = station_ver3['POP_10'] + station_ver3['POP_60']
station_ver3.head(1)

시간(날짜별) 데이터를 위도 경도를 기준으로 각 타겟 인구수의 평균을 구했다.

## 시간대별 유동인구 X 시간대별 통행량 전처리

In [ ]:
# 좌표별 시간대별 유동인구
pop_time = pd.read_csv('14.수원시_시간대별_유동인구(2020).csv')
pop_time = pop_time.groupby(['lon','lat']).mean().drop(['STD_YM'], axis=1)
pop_time = pop_time.reset_index()

pop_time = gpd.GeoDataFrame(pop_time, geometry=gpd.points_from_xy(pop_time.lon, pop_time.lat))
#pop_time = pop_time.drop(['lon','lat'],axis=1)

print(pop_time.shape)
pop_time.head(3)

In [ ]:
# 정류장 주변으로 빈틈이 없도록 사각형 버퍼 생성
def make_square_polygon(gdf, radius = 25):
    
    gdf.crs = 'epsg:4326'
    gdf = gdf.to_crs(epsg=5179)

    buffer = gdf.buffer(radius, cap_style=3)
    gdf['BUFFER'] = buffer.to_crs(epsg=4326)
    gdf = gdf.to_crs(epsg=4326)
    return gdf

pop_time = make_square_polygon(pop_time)
pop_time.head(3)

In [ ]:
pop_time['coordinates'] = pop_time['BUFFER'].apply(lambda x : mapping(x)['coordinates'][0])

In [ ]:
layer1 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    pop_time, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.7
)

center = center
view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1], 
    zoom=11
) 

r = pdk.Deck(layers=[base, layer1], initial_view_state=view_state,
            mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA') 
r.to_html()

In [ ]:
traffic = pd.read_csv('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
traffic = traffic.rename(columns = {'상세도로망_LinkID' : 'link_id', '전체_추정교통량':'추정교통량'})
traffic = traffic[['link_id','시간적범위','추정교통량']]

# 시간적범위 칼럼 데이터타입 통일
traffic = traffic[traffic['시간적범위'] != 'fulltime']
traffic['시간적범위'] = traffic['시간적범위'].apply(pd.to_numeric)
traffic = pd.pivot_table(traffic, index=['link_id'], columns=['시간적범위'], values=['추정교통량'])
traffic.columns = list(range(0,24))
traffic = traffic.reset_index()

traffic['link_id'] = traffic['link_id'].apply(lambda x: int(str(x)[:-2]))
traffic.head(1)

In [ ]:
network = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
network['link_id'] = network['link_id'].apply(pd.to_numeric)
traffic_network = traffic.merge(network, on =['link_id'], how='left')
traffic_network = traffic_network.iloc[:,list(range(1,25))+[-1]]
traffic_network = gpd.GeoDataFrame(traffic_network)

In [ ]:
# 유동인구와 통행량이 미치는 영향 차이가 크지 않도록 스케일링

from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
pop_time.iloc[:,2:26] = Scaler.fit_transform(pop_time.iloc[:,2:26])

Scaler = MinMaxScaler()
traffic_network[list(range(0,24))] = Scaler.fit_transform(traffic_network[list(range(0,24))])
traffic_network.head(2)

In [ ]:
# 도로에 근접한 gird만 추출
inter = []
for i,x in pop_time.iterrows():
    intersected = traffic_network.intersects(x.BUFFER)
    temp = traffic_network[intersected]
    if len(temp) == 0:
        inter.append(False)
    else:
        inter.append(True)
    if i % 10000 == 0:
        print("loading..")
    
pop_time['INTERSECTED'] = inter
pop_time_inter = pop_time[pop_time['INTERSECTED']]

In [ ]:
# 잘 되었나 확인
layer1 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    pop_time_inter, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.7
)

center = center
view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1], 
    zoom=11
) 

r = pdk.Deck(layers=[base, layer1], initial_view_state=view_state,
            mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA') 
r.to_html()

In [ ]:
sum_product = []
for i,x in pop_time_inter.iterrows():
    intersected = traffic_network.intersects(x.BUFFER)
    temp = traffic_network[intersected]
    v1 = x[2:26].values
    v2 = temp[list(range(0,24))].mean(axis=0).values
    sum_product.append(np.sum(v1*v2))
    if i % 3000 == 0:
        print("loading..")
    
pop_time_inter['POPxTRAFFIC'] = sum_product
pop_time_inter.head()

In [ ]:
p90 = np.percentile(pop_time_inter['POPxTRAFFIC'], 90)
m = pop_time_inter

In [ ]:
pop_time_inter['reg'] = pop_time_inter['POPxTRAFFIC'].apply(lambda x: x/p90 if x<=p90 else 1)
pop_time_inter['coordinates'] = pop_time_inter['BUFFER'].apply(lambda x : mapping(x)['coordinates'][0])
pop_time_inter.head()

In [ ]:
layer1 = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    pop_time_inter, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 255*reg, 0]' # 각 데이터 별 rgb 또는 rgba 값 (0~255)
)

center = center
view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1],
    zoom=11
) 

#view_state.bearing = -15
#view_state.pitch = 45

r = pdk.Deck(layers=[base,layer1], initial_view_state=view_state,
            mapbox_key = mapbox_key) 
r.to_html()

In [ ]:
pop_traffic = pop_time_inter[['POPxTRAFFIC','geometry']]
print(pop_traffic.shape)
pop_traffic.head()

In [ ]:
pop_traffic.to_csv('pop_traffic_final.csv', index = False)

# 모델링

### 클러스터링

In [ ]:
# 클러스터링에 사용할 데이터
data = station_ver3[['정류소ID','lon','lat','geometry','BUFFER','총대기시간','POP_vul','먼지유발건물','취약계층건물']]
train_data_1 = data[['POP_vul','총대기시간','먼지유발건물','취약계층건물']]
train_data_1.head()

In [ ]:
!pip install delayed

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import math

def visualize_silhouette(cluster_lists, X_features,a): 
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산.
        if a == 'kmeans' :
            clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state = 0)
        elif a == 'spectral' :
            clusterer = SpectralClustering(n_clusters = n_cluster, random_state = 0)
        elif a == 'agg' :
            clusterer = AgglomerativeClustering(n_clusters = n_cluster)
        elif a == 'gmm' :
            clusterer = GaussianMixture(n_components = n_cluster, random_state = 0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel(a)
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

In [ ]:
scale_1 = StandardScaler()
scale_1.fit(train_data_1)
train_data_scaled_1 = scale_1.transform(train_data_1)
train_data_1[list(train_data_1.columns)] = train_data_scaled_1
train_data_1.head()

In [ ]:
# cluster 개수를 2개, 3개, 4개, 5개 일때의 클러스터별 실루엣 계수 평균값을 시각화 
#visualize_silhouette([2,3,4,5,6], train_data_1)
visualize_silhouette([4,5,6,7,8], train_data_1,'kmeans')

### K-means
---

In [ ]:
data['KMEANS'] =  KMeans(n_clusters = 4, init = 'k-means++', max_iter = 100, random_state = 0).fit_predict(train_data_1)
test = data.groupby(['KMEANS']).mean()
score_kmeans = np.zeros(len(test))
for i in list(train_data_1.columns) :
    score_kmeans += ss.rankdata(test[i])
score_kmeans = pd.Series(score_kmeans).sort_values()
print('---스코어---')
print(score_kmeans)
print('--군집별 정류장 개수--')
print(data['KMEANS'].value_counts())
drop_kmeans = list(score_kmeans.index[:2])
print('--제거할 군집--')
print(drop_kmeans)
test.head()

### Spectral Clustering
---

In [ ]:
data['Spectral'] = SpectralClustering(n_clusters = 7, random_state = 0).fit_predict(train_data_1)
test = data.groupby(['Spectral']).mean()
score_spectral = np.zeros(len(test))
for i in list(train_data_1.columns) :
    score_spectral += ss.rankdata(test[i])
score_spectral = pd.Series(score_spectral)
print('---스코어---')
print(score_spectral.sort_values())
print('--군집별 정류장 개수--')
print(data['Spectral'].value_counts())
# 0과 6중 스코어가 동일하므로 다른 클러스터링과의 공통부분 결합을 위해 6를 직접 지정
#drop_spectral = list(score_spectral.sort_values().index[:2])
print('--제거할 군집--')
drop_spectral = [2,6]
print(drop_spectral)
test

### Agglomerative Clustering
---

In [ ]:
data['Agg'] =  AgglomerativeClustering(n_clusters = 5).fit_predict(train_data_1)
test = data.groupby(['Agg']).mean()
score_agg = np.zeros(len(test))
for i in list(train_data_1.columns) :
    score_agg += ss.rankdata(test[i])
score_agg = pd.Series(score_agg)
print('---스코어---')
print(score_agg.sort_values())
print('--군집별 정류장 개수--')
print(data['Agg'].value_counts())
drop_agg = list(score_agg.sort_values().index[:2])
print(drop_agg)
test

### GMM
---

In [ ]:
data['GMM'] = GaussianMixture(n_components = 5, random_state = 0).fit_predict(train_data_1)
test = data.groupby(['GMM']).mean()
score_GMM = np.zeros(len(test))
for i in list(train_data_1.columns) :
    score_GMM += ss.rankdata(test[i])
score_GMM = pd.Series(score_GMM)
print('---스코어---')
print(score_GMM.sort_values())
print('--군집별 정류장 개수--')
print(data['GMM'].value_counts())
drop_GMM = list(score_GMM.sort_values().index[:2])
print(drop_GMM)
test

### 각 클러스터링 알고리즘의 군집 결과에서 공통 데이터 추출

In [ ]:
clustered_data = data[np.array(data.KMEANS != drop_kmeans[0]) & np.array(data.KMEANS != drop_kmeans[1])
    & np.array(data.GMM != drop_GMM[0]) & np.array(data.GMM != drop_GMM[1])
    & np.array(data.Spectral != drop_spectral[0]) & np.array(data.Spectral != drop_spectral[1])
    & np.array(data.Agg != drop_agg[0]) & np.array(data.Agg != drop_agg[1])]
len(clustered_data)

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  clustered_data, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,160,0]',
                  get_radius=100,
                  pickable=True, auto_highlight=True 
                 ) 

# [100,255*전체 승차 건수_reg,100]

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat1], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

### MCLP

In [ ]:
pop_traffic = pd.read_csv('pop_traffic_final.csv')
pop_traffic

In [ ]:
station = clustered_data
station = station[['정류소ID','lon', 'lat', 'geometry']]
station = gpd.GeoDataFrame(station, geometry=gpd.points_from_xy(station.lon, station.lat))
#station['dv'] = ['y[%s]'%s for s in station.정류소ID ]
station = station.reset_index()
station.head()

In [ ]:
if not isinstance(pop_traffic.geometry, gpd.geoseries.GeoSeries):
    pop_traffic['lon'] = pop_traffic['geometry'].apply(lambda x: x[7:-1].split(' ')[0])
    pop_traffic['lat'] = pop_traffic['geometry'].apply(lambda x: x[7:-1].split(' ')[1])
    pop_traffic =  gpd.GeoDataFrame(pop_traffic, geometry=gpd.points_from_xy(pop_traffic.lon, pop_traffic.lat))
    pop_traffic = pop_traffic.drop(['lon','lat'], axis=1)
pop_traffic.reset_index(inplace=True)
pop_traffic = pop_traffic.rename(columns = {'POPxTRAFFIC' : 'weights'})
pop_traffic.head()

In [ ]:
pop_traffic.crs = 'epsg:4326'
station.crs = 'epsg:4326'

pop_traffic = pop_traffic.to_crs(epsg=5179) 
station = station.to_crs(epsg=5179)

In [ ]:
# 각 정류장과 기준 좌표들 간의 거리 행렬
d = np.empty((len(pop_traffic), len(station)))
for i in range(len(pop_traffic)):
    point_i = pop_traffic.geometry[i]
    d[i] = [point_i.distance(point_j) for point_j in station.geometry]
    if i % 2000 == 0:
        print('loading'+ '.'*(i//2000)+ '_'*(6-i//2000))
d.shape

In [ ]:
# 정류장사이의 거리
# 정류장 간에 최소 거리가 어느 정도 이상이어야 한다는 제약조건을 반영하기 위함.
r = np.empty((len(station), len(station)))
for i in range(len(station)):
    station_i = station.geometry[i]
    r[i] = [ station_i.distance(station_j) for station_j in station.geometry]
r.shape

In [ ]:
I = pop_traffic.index.values
J = station.index.values
S = 400
min_dist = 50

a = pop_traffic.weights.values
P = 30

# Compute the sets Ni
# NB: this will be a list in which each item is a list of nodes
# within the threshold distance of the i'th node
N = [[j for j in J if d[i][j] < S] for i in I]
R = [[int(r[station_i][station_j] < min_dist) for station_j in J ] for station_i in J]
# Formulate optimisation

prob = LpProblem("MCLP", LpMaximize)
x = LpVariable.dicts("x", J, lowBound=0, upBound=1, cat='Integer')
y = LpVariable.dicts("y", I, lowBound=0, upBound=1, cat='Integer')

# Objective
prob += lpSum([a[i]*y[i] for i in I])

# Constraints
for i in I:
    prob += lpSum([x[j] for j in N[i]]) >= y[i]
for j in J:
    prob += lpSum([x[rr] for rr in R[j]]) <= 1
    

prob += lpSum([x[j] for j in J]) == P

# Solve problem
prob.solve()

x_soln = np.array([x[j].varValue for j in J])

# And print some output
print (("Status:"), LpStatus[prob.status])
print ("Weight Covered is = ", value(prob.objective))
print ("x = ", x_soln)

In [ ]:
station['opt'] = x_soln
station.head()

In [ ]:
opt_id = station[station['opt'] == 1]['정류소ID'].values
station_raw = pd.read_csv('1.수원시_버스정류장.csv')
station_raw['optimal'] = station_raw['정류장ID'].apply(lambda x: x in opt_id)
station_raw[station_raw['optimal'] == True]

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  station_raw[station_raw['optimal']==0], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[0,255,0]',
                  get_radius=50,
                  opacity = 0.1,
                  pickable=True, auto_highlight=True
                 ) 

scat2 = pdk.Layer('ScatterplotLayer',
                  station_raw[station_raw['optimal']==1], 
                  get_position = ['lon','lat'], 
                  get_fill_color='[127,255,212]',
                  get_radius=200,
                  pickable=True, auto_highlight=True 
                 ) 

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat1, scat2], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

## 선정 결과에서 접근성 및 편의성 확인
- 지하철역의 위치정보

In [ ]:
subway = pd.read_csv('8.수원시_지하철역_위치정보.csv')
subway = subway.rename(columns = {'역사명':'역명', '노선명':'호선명'})
subway.head()

In [ ]:
landmark_dict = {'랜드마크':['광교호수공원', '국토지리정보원지도박물관','만석공원','방화수류정',
                        '화성행궁', '서호공원', '광교저수지쉼터', '롯데백화점 수원점',
                        'NC백화점 수원터미널점', 'AK플라자 수원점', '갤러리아백화점 광교'],
                'lon':[127.0637281,127.0529836, 126.9989156, 127.015922,
                      127.0115454,126.9882966,127.0105735,126.9950386,
                      127.0180337,126.9981621,127.0527941],
                'lat':[37.283123, 37.2761374, 37.3007055,37.2875274,
                      37.281962,37.2803729,37.3068497,37.264328,
                      37.2500302,37.2654804,37.2805385]}
landmark = pd.DataFrame(landmark_dict)
landmark

In [ ]:
scat3 = pdk.Layer('ScatterplotLayer',
                  subway, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[255,100,0]',
                  get_radius=150,
                  pickable=True, auto_highlight=True
                 ) 

scat4 = pdk.Layer('ScatterplotLayer',
                  landmark, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[255,255,0]',
                  get_radius=150,
                  pickable=True, auto_highlight=True
                 )  

center = center 
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 
r = pdk.Deck(layers=[base, scat1, scat2, scat3, scat4], initial_view_state=view_state,
            mapbox_key = "pk.eyJ1IjoiamFzb243NjU1IiwiYSI6ImNrbTF6ODV2ajBubHgydm05Y2dsZmhnbXoifQ.HqZQtFRrorSuizkwe6kpFg") 
r.to_html()

# 최종 입지선정 결과

In [ ]:
pop = pop_gender.drop(['lon','lat','TOTAL'], axis=1)
column_list = [f'{age}대 남성' for age in range(10, 60, 10)] + ['60대 이상 남성'] 
column_list += [f'{age}대 여성 ' for age in range(10, 60, 10)] + ['60대 이상 여성'] 
pop.columns = column_list + ['geometry']
pop.crs = 'epsg:4326'
pop.head(3)

In [ ]:
0.92/0.77

In [ ]:
# 논문 "광고유형 및 성별에 따른 광고효과와 정서반응 패턴, 정은경 외, 2012"에 따르면 여성은 약 1.2배 더 큰 광고 효과를 가짐
for col in [f'{age}대 여성 ' for age in range(10, 60, 10)] + ['60대 이상 여성']:
    pop[col] = pop[col]*(0.92/0.77)

In [ ]:
station_ver4 = station_ver3[['정류소ID', '정류장명','lon','lat','BUFFER']]
station_ver4['optimal'] = station_ver4['정류소ID'].apply(lambda x: x in opt_id)
station_ver4 = station_ver4[station_ver4['optimal'] == True]

for col in tqdm(column_list):
    station_ver4[col] = station_ver4['BUFFER'].apply(lambda x: pop_within_buffer(pop, col, x))
station_ver4.sample(1)

In [ ]:
#광고 타겟층 선정
advertise_target = []
for i in range(len(station_ver4)):
    target_list = station_ver4[column_list].iloc[i,:]
    advertise_target.append(target_list.idxmax())
    
station_ver4['광고 Target층'] = advertise_target
station_final = station_ver4.drop(['optimal'] + column_list, axis=1)
station_final.head(1)

In [ ]:
pop_traffic = pd.read_csv('pop_traffic_final.csv')

if not isinstance(pop_traffic.geometry, gpd.geoseries.GeoSeries):
    pop_traffic['lon'] = pop_traffic['geometry'].apply(lambda x: x[7:-1].split(' ')[0])
    pop_traffic['lat'] = pop_traffic['geometry'].apply(lambda x: x[7:-1].split(' ')[1])
    pop_traffic =  gpd.GeoDataFrame(pop_traffic, geometry=gpd.points_from_xy(pop_traffic.lon, pop_traffic.lat))
    pop_traffic = pop_traffic.drop(['lon','lat'], axis=1)
pop_traffic.reset_index(inplace=True)
pop_traffic.crs = 'epsg:4326'
pop_traffic.head(3)

In [ ]:
# 선정된 POP x TRAFFIC 기준으로 순위 산정
station_final['order'] = station_final['BUFFER'].apply(lambda x: pop_within_buffer(pop_traffic, 'POPxTRAFFIC', x))
station_final = station_final.sort_values(['order'],ascending= False).reset_index()
station_final = station_final.drop(['BUFFER','order'], axis=1)

station_final['index'] = range(1, 31)
station_final.rename(columns = {'정류소ID':'정류장ID','index' : '설치순위', 'lon':'X축좌표(경도)', 'lat':'Y축좌표(위도)'}, inplace = True)
station_final = station_final[['설치순위','정류장명','정류장ID','X축좌표(경도)','Y축좌표(위도)','광고 Target층']]
station_final

In [ ]:
station_final.to_csv('분석결과_수타버스.csv')

# 잡동사니

## 취약계층(인구정보)

from shapely.geometry import Polygon

older = gpd.read_file('17.수원시_인구정보(고령)_격자.geojson')
older['geometry'] = older['geometry'].apply(lambda x: x.centroid)
older.head()

## 반경 내에 취약계층(older) 평균 추가
import shapely.speedups
shapely.speedups.enable()
from shapely.geometry import Point

def pop_within_buffer(df, column, poly):
    in_out = df.within(poly)
    within_buffer = df[in_out]
    return np.mean(within_buffer[column])

station_ver1['older'] = station_ver1['BUFFER'].apply(lambda x: pop_within_buffer(older, 'val', x))
station_ver1.head()

## 유소년에 대해서도 똑같이 해줌
youth = gpd.read_file('19.수원시_인구정보(유소년)_격자.geojson')
youth['geometry'] = youth['geometry'].apply(lambda x: x.centroid)

station_1['youth'] = station_1['youth'].apply(lambda x: pop_within_buffer(youth, 'val', x))
station_1.head()